
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/sample.json"
file_type = "json"

# CSV options
infer_schema = "TRUE"
first_row_is_header = "TRUE"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

_corrupt_record,emp_id,name,projects
[,null,null,null
null,1,Alice,"List(List(30, Alpha), List(10, Beta))"
null,2,Bob,"List(List(25, Alpha), List(15, Gamma))"
null,3,Charlie,"List(List(20, Alpha))"
null,4,Diana,"List(List(35, Delta), List(20, Alpha))"
null,5,Ethan,"List(List(40, Beta))"
null,6,Fiona,"List(List(22, Gamma))"
null,7,George,"List(List(28, Beta), List(12, Delta))"
null,8,Hannah,"List(List(16, Alpha))"
null,9,Ivan,"List(List(18, Gamma), List(8, Beta))"


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import array,array_contains,array_position,array_remove,col,expr
df1=df.select("name",array(col("name")).alias("array_name"))
df1.show()


+-------+----------+
|   name|array_name|
+-------+----------+
|   null|    [null]|
|  Alice|   [Alice]|
|    Bob|     [Bob]|
|Charlie| [Charlie]|
|  Diana|   [Diana]|
|  Ethan|   [Ethan]|
|  Fiona|   [Fiona]|
| George|  [George]|
| Hannah|  [Hannah]|
|   Ivan|    [Ivan]|
|   Jill|    [Jill]|
|  Kevin|   [Kevin]|
|  Laura|   [Laura]|
|   Mike|    [Mike]|
|   Nina|    [Nina]|
|  Oscar|   [Oscar]|
|    Pam|     [Pam]|
|Quentin| [Quentin]|
| Rachel|  [Rachel]|
|  Steve|   [Steve]|
+-------+----------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import expr
df2=df.withColumn("alpha",expr("array_contains(transform(projects, x -> x.project_name), 'Alpha')"))\
      .withColumn("Beta",expr("array_contains(transform(projects, x ->x.project_name),'Beta')"))\
      .withColumn("Gamma",expr("array_contains(transform(projects,x->x.project_name),'Gamma')")) 
df2.show()       

df3=df2.select("projects","alpha","Beta","Gamma")
df3.show(truncate=False)
            
    
  

+---------------+------+-------+--------------------+-----+-----+-----+
|_corrupt_record|emp_id|   name|            projects|alpha| Beta|Gamma|
+---------------+------+-------+--------------------+-----+-----+-----+
|              [|  null|   null|                null| null| null| null|
|           null|     1|  Alice|[{30, Alpha}, {10...| true| true|false|
|           null|     2|    Bob|[{25, Alpha}, {15...| true|false| true|
|           null|     3|Charlie|       [{20, Alpha}]| true|false|false|
|           null|     4|  Diana|[{35, Delta}, {20...| true|false|false|
|           null|     5|  Ethan|        [{40, Beta}]|false| true|false|
|           null|     6|  Fiona|       [{22, Gamma}]|false|false| true|
|           null|     7| George|[{28, Beta}, {12,...|false| true|false|
|           null|     8| Hannah|       [{16, Alpha}]| true|false|false|
|           null|     9|   Ivan|[{18, Gamma}, {8,...|false| true| true|
|           null|    10|   Jill|       [{45, Delta}]|false|false

In [0]:
from pyspark.sql.functions import size
df_len=df2.withColumn("projects_count",size("projects").alias("proj_len"))
df_len.show()
df_size=df_len.select("projects","projects_count")
df_size.show()

+---------------+------+-------+--------------------+-----+-----+-----+--------------+
|_corrupt_record|emp_id|   name|            projects|alpha| Beta|Gamma|projects_count|
+---------------+------+-------+--------------------+-----+-----+-----+--------------+
|              [|  null|   null|                null| null| null| null|            -1|
|           null|     1|  Alice|[{30, Alpha}, {10...| true| true|false|             2|
|           null|     2|    Bob|[{25, Alpha}, {15...| true|false| true|             2|
|           null|     3|Charlie|       [{20, Alpha}]| true|false|false|             1|
|           null|     4|  Diana|[{35, Delta}, {20...| true|false|false|             2|
|           null|     5|  Ethan|        [{40, Beta}]|false| true|false|             1|
|           null|     6|  Fiona|       [{22, Gamma}]|false|false| true|             1|
|           null|     7| George|[{28, Beta}, {12,...|false| true|false|             2|
|           null|     8| Hannah|       [{16

In [0]:
from pyspark.sql.functions import array_remove,expr
df_remove=df_size.withColumn ("projects_without_alpha",
    expr("filter(projects, x -> x.project_name != 'Alpha')")
)
df1_remove=df_remove.select("projects_without_Alpha","projects").show(truncate=False)

+--------------------------+--------------------------+
|projects_without_Alpha    |projects                  |
+--------------------------+--------------------------+
|null                      |null                      |
|[{10, Beta}]              |[{30, Alpha}, {10, Beta}] |
|[{15, Gamma}]             |[{25, Alpha}, {15, Gamma}]|
|[]                        |[{20, Alpha}]             |
|[{35, Delta}]             |[{35, Delta}, {20, Alpha}]|
|[{40, Beta}]              |[{40, Beta}]              |
|[{22, Gamma}]             |[{22, Gamma}]             |
|[{28, Beta}, {12, Delta}] |[{28, Beta}, {12, Delta}] |
|[]                        |[{16, Alpha}]             |
|[{18, Gamma}, {8, Beta}]  |[{18, Gamma}, {8, Beta}]  |
|[{45, Delta}]             |[{45, Delta}]             |
|[{6, Beta}]               |[{32, Alpha}, {6, Beta}]  |
|[{30, Gamma}]             |[{30, Gamma}]             |
|[]                        |[{10, Alpha}]             |
|[{25, Delta}, {10, Gamma}]|[{25, Delta}, {10, G

In [0]:
from pyspark.sql.functions import array_position,expr
df_position=df_size.withColumn("Alpha_index",
                               expr("array_position(transform(projects, x -> x.project_name),'Alpha')")
)
df_index=df_position.select("projects","Alpha_index").show(truncate=False)

+--------------------------+-----------+
|projects                  |Alpha_index|
+--------------------------+-----------+
|null                      |null       |
|[{30, Alpha}, {10, Beta}] |1          |
|[{25, Alpha}, {15, Gamma}]|1          |
|[{20, Alpha}]             |1          |
|[{35, Delta}, {20, Alpha}]|2          |
|[{40, Beta}]              |0          |
|[{22, Gamma}]             |0          |
|[{28, Beta}, {12, Delta}] |0          |
|[{16, Alpha}]             |1          |
|[{18, Gamma}, {8, Beta}]  |0          |
|[{45, Delta}]             |0          |
|[{32, Alpha}, {6, Beta}]  |1          |
|[{30, Gamma}]             |0          |
|[{10, Alpha}]             |1          |
|[{25, Delta}, {10, Gamma}]|0          |
|[{22, Beta}]              |0          |
|[{17, Alpha}]             |1          |
|[{24, Gamma}]             |0          |
|[{19, Delta}]             |0          |
|[{26, Alpha}, {14, Beta}] |1          |
+--------------------------+-----------+
only showing top

In [0]:
from pyspark.sql.functions import explode,posexplode,explode_outer,posexplode_outer
df_explode=df_len.withColumn("projects_",explode("projects").alias("project_explode"))
df_explode.show()
#from pyspark.sql.functions import explode,posexplode,explode_outer,posexplode_outer
#df_posex=df_len.withColumn("projects",posexplode("projects").alias("pro_ex"))
#df_posex1 = df_posex.select("projects.pos", "projects.col.hours", "projects.col.project_name")
#df_posex1.show()

                           
                





+---------------+------+-------+--------------------+-----+-----+-----+--------------+-----------+
|_corrupt_record|emp_id|   name|            projects|alpha| Beta|Gamma|projects_count|  projects_|
+---------------+------+-------+--------------------+-----+-----+-----+--------------+-----------+
|           null|     1|  Alice|[{30, Alpha}, {10...| true| true|false|             2|{30, Alpha}|
|           null|     1|  Alice|[{30, Alpha}, {10...| true| true|false|             2| {10, Beta}|
|           null|     2|    Bob|[{25, Alpha}, {15...| true|false| true|             2|{25, Alpha}|
|           null|     2|    Bob|[{25, Alpha}, {15...| true|false| true|             2|{15, Gamma}|
|           null|     3|Charlie|       [{20, Alpha}]| true|false|false|             1|{20, Alpha}|
|           null|     4|  Diana|[{35, Delta}, {20...| true|false|false|             2|{35, Delta}|
|           null|     4|  Diana|[{35, Delta}, {20...| true|false|false|             2|{20, Alpha}|
|         

In [0]:
from pyspark.sql.functions import to_json



df_json = df.withColumn("projects_json", to_json(col("projects")))
df_json.select("projects_json").show(truncate=False)

+-------------------------------------------------------------------------+
|projects_json                                                            |
+-------------------------------------------------------------------------+
|null                                                                     |
|[{"hours":30,"project_name":"Alpha"},{"hours":10,"project_name":"Beta"}] |
|[{"hours":25,"project_name":"Alpha"},{"hours":15,"project_name":"Gamma"}]|
|[{"hours":20,"project_name":"Alpha"}]                                    |
|[{"hours":35,"project_name":"Delta"},{"hours":20,"project_name":"Alpha"}]|
|[{"hours":40,"project_name":"Beta"}]                                     |
|[{"hours":22,"project_name":"Gamma"}]                                    |
|[{"hours":28,"project_name":"Beta"},{"hours":12,"project_name":"Delta"}] |
|[{"hours":16,"project_name":"Alpha"}]                                    |
|[{"hours":18,"project_name":"Gamma"},{"hours":8,"project_name":"Beta"}]  |
|[{"hours":4